# Auto Generated Agent Chat: Multi Task Chats

AutoGen offers conversable LLM agents, which can be used to solve various tasks with human or automatic feedback, including tasks that require using tools via code.
Please find documentation about this feature [here](https://microsoft.github.io/autogen/docs/Use-Cases/agent_chat).


## Requirements

AutoGen requires `Python>=3.8`. To run this notebook example, please install:
```bash
pip install pyautogen
```

## Example Task



In [24]:
from autogen import AssistantAgent, UserProxyAgent, config_list_from_json

config_list = config_list_from_json(env_or_file="OAI_CONFIG_LIST")

financial_tasks = [
    """What are the current stock prices of NVDA and TESLA, and how is the performance over the past month in terms of percentage change? """,
    """Investigate the possible reasons of the stock performance.""",
]

writing_tasks = ["""Develop an engaging blog post using any information provided."""]

financial_assistant = AssistantAgent(
    name="Financial_assistant",
    llm_config={"config_list": config_list},
)
research_assistant = AssistantAgent(
    name="Researcher",
    llm_config={"config_list": config_list},
)
writer = AssistantAgent(
    name="writer",
    llm_config={"config_list": config_list},
    system_message="""
        You are a professional writer, known for
        your insightful and engaging articles.
        You transform complex concepts into compelling narratives.
        Reply "TERMINATE" in the end when everything is done.
        """,
)

user = UserProxyAgent(
    name="User",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": "tasks",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)

user.initiate_chats(
    [
        {
            "recipient": financial_assistant,
            "message": financial_tasks[0],
            "clear_history": True,
            "silent": False,
            "get_takeaway": "last_msg",
        },
        {
            "recipient": research_assistant,
            "message": financial_tasks[1],
            "get_takeaway": "last_msg",
        },
        {
            "recipient": writer,
            "message": writing_tasks[0],
            "get_takeaway": "last_msg",
        },
    ]
)

User (to Financial_assistant):


    What are the current stock prices of NVDA and TESLA, and how is the performance over the past month in terms of percentage change? 

--------------------------------------------------------------------------------
Financial_assistant (to User):

To get the current stock prices of NVIDIA Corporation (NVDA) and Tesla, Inc. (TSLA), as well as their performance over the past month in terms of percentage change, we can use financial APIs or web scraping from financial websites that provide real-time stock data.

For this task, I'll use the `yfinance` library in Python which allows fetching historical market data from Yahoo Finance. First, you will need to install `yfinance` if it is not already installed. Then, we can write a Python script to get the current price and calculate the percentage change over the past month.

Let's proceed with the Python code to fetch this data.

```python
# filename: stock_info.py

import yfinance as yf
from datetime import

Chats in a Group Chat Setting



In [1]:
from autogen import AssistantAgent, UserProxyAgent, GroupChatManager, GroupChat, config_list_from_json

config_list = config_list_from_json(env_or_file="OAI_CONFIG_LIST")

financial_tasks = [
    """What are the current stock prices of NVDA and TESLA, and how is the performance over the past month in terms of percentage change? """,
    """Investigate the possible reasons of the stock performance.""",
]

writing_tasks = [
    """Develop an engaging blog
        post using any information provided."""
]

user_proxy = UserProxyAgent(
    name="User_proxy",
    system_message="A human admin.",
    # code_execution_config={"last_n_messages": 3, "work_dir": "groupchat"},
    human_input_mode="NEVER",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": "groupchat",
        "use_docker": False,
    },
)

research_assistant = AssistantAgent(
    name="Researcher",
    llm_config={"config_list": config_list},
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
)

writer = AssistantAgent(
    name="Writer",
    llm_config={"config_list": config_list},
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    system_message="""
    You are a professional writer, known for
    your insightful and engaging articles.
    You transform complex concepts into compelling narratives.
    Reply "TERMINATE" in the end when everything is done.
    """,
)

critic = AssistantAgent(
    name="Critic",
    system_message="""Critic. Double check plan, claims, code from other agents and provide feedback. Check whether the plan includes adding verifiable info such as source URL.
    Reply "TERMINATE" in the end when everything is done.
    """,
    llm_config={"config_list": config_list},
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
)

groupchat_1 = GroupChat(agents=[user_proxy, research_assistant, critic], messages=[], max_round=50)

groupchat_2 = GroupChat(agents=[user_proxy, writer, critic], messages=[], max_round=50)

manager_1 = GroupChatManager(
    groupchat=groupchat_1,
    name="Research_manager",
    llm_config={"config_list": config_list},
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": "groupchat",
        "use_docker": False,
    },
)
manager_2 = GroupChatManager(
    groupchat=groupchat_2,
    name="Writing_manager",
    llm_config={"config_list": config_list},
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": "groupchat",
        "use_docker": False,
    },
)

user = UserProxyAgent(
    name="User",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": "tasks",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)
user.initiate_chats(
    [
        {"recipient": research_assistant, "message": financial_tasks[0], "get_takeaway": "last_msg"},
        {"recipient": manager_1, "message": financial_tasks[1], "get_takeaway": "last_msg"},
        {"recipient": manager_2, "message": writing_tasks[0], "get_takeaway": "last_msg"},
    ]
)

User (to Researcher):

What are the current stock prices of NVDA and TESLA, and how is the performance over the past month in terms of percentage change? 

--------------------------------------------------------------------------------
Researcher (to User):

To get the current stock prices of NVDA (NVIDIA Corporation) and TSLA (Tesla, Inc.), as well as their performance over the past month in terms of percentage change, we need to access financial data. This can be done using various financial data APIs or Python libraries that provide access to such information.

I will use the `yfinance` library in Python, which enables us to scrape historical market data from Yahoo Finance and calculate the performance metrics. If `yfinance` is not installed on your system, you can install it using pip.

Here are the steps we will follow:
1. Install the `yfinance` library using pip if necessary.
2. Fetch the current stock prices and historical prices for the past month for NVDA and TSLA.
3. Calcula